In [1]:
from pathlib import Path
import json
import IPython.display as ipd

import torch
from torch import Tensor

from hifigan.models import Generator as HiFiGAN
from hifigan.utils import AttrDict

In [2]:
# load hifigan
device = "cpu"
hifigan_dir = Path("/Users/cafr02/repos/spkanon/checkpoints/knnvc")
config = json.load(open(hifigan_dir/"hifigan.json"))
h = AttrDict(config)
hifigan = HiFiGAN(h).to(device)

hifigan_ckpt = torch.load(
    "/Users/cafr02/repos/spkanon/hifigan_logs/g_00120000.pt", map_location="cpu"
)
# hifigan_ckpt = torch.load(hifigan_dir/"hifigan.pt", map_location=device)
hifigan.load_state_dict(hifigan_ckpt["generator"])
hifigan.eval()
hifigan.remove_weight_norm()

/Users/cafr02/repos/train_vocoder/venv/lib/python3.11/site-packages/torch/nn/utils/weight_norm.py:144: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


Removing weight norm...


In [3]:
prematch_dir = Path("ls_test_clean_prematch")
n_spk = 3
n_utts_per_spk = 1

spk_dirs = sorted([p for p in prematch_dir.iterdir() if p.is_dir()])[:n_spk]

for spk_dir in spk_dirs:
    pt_files = sorted(spk_dir.glob("*/*.pt"))[:n_utts_per_spk]
    for pt in pt_files:
        
        print(pt)
        feats = torch.load(pt).float().unsqueeze(0)
        with torch.inference_mode():
            out_wav = hifigan(feats)

        ipd.display(ipd.Audio(out_wav.squeeze().numpy(), rate=16000))

ls_test_clean_prematch/1089/134686/1089-134686-0000.pt


ls_test_clean_prematch/1188/133604/1188-133604-0000.pt
